# Shutting Ports with no Traffic

## For single unit switch

In [ ]:
import re
import exsh

# Run the EXOS command and capture output
try:
    output = exsh.clicmd("show ports statistics no-refresh", capture=True)
except Exception as e:
    print(f"[-] Error running command: {e}")
    exit()

# Regex to extract port numbers and traffic stats (including zero values)
interfaces = re.findall(r"(\d+)\s+\S+\s+(\d+)\s+(\d+)", output)

inactive_ports = []

# Find inactive interfaces, excluding ports 49 and above
for interface, tx_pkts, rx_pkts in interfaces:
    port_num = int(interface)  # Convert to integer for comparison
    tx_pkts, rx_pkts = int(tx_pkts), int(rx_pkts)

    if port_num < 49 and tx_pkts == 0 and rx_pkts == 0:  # Exclude ports 49+
        inactive_ports.append(interface)

# Generate and execute EXOS CLI commands
if inactive_ports:
    for port in inactive_ports:
        if port == '0':
            continue
        try:
            exsh.clicmd(f"disable ports {port}")
            print(f"[+] Successfully disabled port {port}")
        except Exception as e:
            print(f"[-] Error disabling port {port}: {e}")
else:
    print("[+] No inactive ports found")

## For switches mounted in stack

In [ ]:
import re
import exsh

# Run the EXOS command and capture output
try:
    output = exsh.clicmd("show ports statistics no-refresh", capture=True)
except Exception as e:
    print(f"[-] Error running command: {e}")
    exit()

# Regex to extract stacked port numbers (format: slot:port) and traffic stats
interfaces = re.findall(r"(\d+:\d+)\s+\S+\s+(\d+)\s+(\d+)", output)

inactive_ports = []

# Find inactive interfaces, excluding ports 49 and above per slot
for interface, tx_pkts, rx_pkts in interfaces:
    slot, port = map(int, interface.split(":"))  # Split into slot and port
    tx_pkts, rx_pkts = int(tx_pkts), int(rx_pkts)

    if port < 49 and tx_pkts == 0 and rx_pkts == 0:  # Exclude ports 49+ per slot
        inactive_ports.append(interface)

# Generate and execute EXOS CLI commands
if inactive_ports:
    for port in inactive_ports:
        try:
            exsh.clicmd(f"disable ports {port}")
            print(f"[+] Successfully disabled port {port}")
        except Exception as e:
            print(f"[-] Error disabling port {port}: {e}")
else:
    print("[+] No inactive ports found")
